# Text classification of tweets:

In [1]:
from __future__ import print_function
import numpy as np
import gensim
from gensim.models import Word2Vec, Doc2Vec, FastText
import gensim.parsing.preprocessing as prep
import csv
import random
#import pandas as pd
#import spacy
#import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier

from numba import jit
import numba as nb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import datasets, svm
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.layers import Conv1D, GlobalMaxPooling1D

print('starting')

C:\Users\Magnu\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


starting


In [2]:
def open_file(fileName):
    with open(str(fileName), "r", encoding="utf8") as sample:
        s = sample.readlines() 
    return s

def create_X(list_of_tweets, w2v, features):
    
    # this function needs some love!
    
    X = np.zeros((len(list_of_tweets),features)) 
    
    for indeks, tweet in enumerate(list_of_tweets):
        for word in tweet:
            try:
                X[indeks,:] = X[indeks,:] + model_tot.wv[str(word)]
            except:
                pass
        N = len(tweet)
        if N>0:
            X[indeks] = X[indeks]/N
    return X

def createWordEmbedding(list_of_tweets, features, epoc, sg=0):
    print(sg)
    model = Word2Vec(list_of_tweets, size=features, window=5, min_count=1, workers=4, sg=sg)
    print('word embedding created, start training:')
    model.train(list_of_tweets, total_examples=len(list_of_tweets), epochs=epoc)
    return model

def createSentEmbedding(list_of_tweets, features, epoc):
    model = Doc2Vec(list_of_tweets, size=features, min_count=1)
    model.train(list_of_tweets, total_examples=len(list_of_tweets), epochs=epoc)
    return model

def save_csv(fileName, test_y):
    ids = np.arange(len(test_y))  
    with open(fileName, 'w') as csvfile:
        tempwriter = csv.writer(csvfile)
        tempwriter.writerow(["Id","Prediction"])
        count = 0
        for row in test_y:
            if row == 0:
                row = -1
            tempwriter.writerow([(ids[count])+1,str(row)])
            count = count + 1
            
def train(method, x, y, x_test):
    met = method.fit(x,y)
    test_y = met.predict(x_test)
    return test_y

def shuffle_tweets(listTweets, y):
    c = list(zip(listTweets, y))
    random.shuffle(c)
    listTweets, y = zip(*c)
    return listTweets, y

def processTrainingData(list_of_tweets):
    # Does not seem to work:
    #list_of_tweets = list(set(list_of_tweets)) # remove duplicate lines, should not be done for test-data (reorders the list)
    #list_of_tweets = [prep.strip_multiple_whitespaces(line) for line in list_of_tweets]
    
    # May work, no clear improvement:
    #list_of_tweets = [prep.strip_tags(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_numeric(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_non_alphanum(line) for line in list_of_tweets]
    #list_of_tweets = [prep.split_alphanum(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_punctuation(line) for line in list_of_tweets]
    #list_of_tweets = [prep.remove_stopwords(line) for line in list_of_tweets]
    #list_of_tweets = [prep.stem_text(line) for line in list_of_tweets]
    
    # this seems to improve the result:
    list_of_tweets = [prep.strip_short(line) for line in list_of_tweets]
    
    # Only one of these can be run simultaneously:
    list_of_tweets = prep.preprocess_documents(list_of_tweets)
    #list_of_tweets = [prep.preprocess_string(line) for line in list_of_tweets]
    #list_of_tweets = [gensim.utils.simple_preprocess(line) for line in list_of_tweets] # simple preprocessing
    return list_of_tweets

def easyProcess(list_of_tweets):
    
    #list_of_tweets = [prep.strip_multiple_whitespaces(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_tags(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_numeric(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_non_alphanum(line) for line in list_of_tweets]
    #list_of_tweets = [prep.split_alphanum(line) for line in list_of_tweets]
    #list_of_tweets = [prep.strip_punctuation(line) for line in list_of_tweets]
    #list_of_tweets = [prep.remove_stopwords(line) for line in list_of_tweets]
    #list_of_tweets = [prep.stem_text(line) for line in list_of_tweets]
    # this seems to improve the result:
    list_of_tweets = [prep.strip_short(line) for line in list_of_tweets]

    return list_of_tweets

print('all functions ready')

all functions ready


In [3]:
positive_tweets = open_file("train_pos.txt")
negative_tweets = open_file("train_neg.txt")

#list_all_tweets = positive_tweets+negative_tweets

positive_tweets = processTrainingData(positive_tweets)

negative_tweets = processTrainingData(negative_tweets)
y = [1]*len(positive_tweets)+[0]*len(negative_tweets)
print(len(y))

#list_all_tweets, y_full = shuffle_tweets(list_all_tweets, y)


all_tweets = positive_tweets+negative_tweets #list of tweets
all_tweets, y = shuffle_tweets(all_tweets, y)
print('loaded and processed short training set')

200000
loaded and processed short training set


In [4]:
#print(list_all_tweets[:5])

Create word embedding based on full set:

In [ ]:

#Creates word embeddings for full training set:

positive_tweets_full = open_file("train_pos_full.txt")
negative_tweets_full = open_file("train_neg_full.txt")
print('all loaded')
positive_tweets_full = processTrainingData(positive_tweets_full)
negative_tweets_full = processTrainingData(negative_tweets_full)
y_full = [1]*len(positive_tweets_full)+[0]*len(negative_tweets_full)
print('finished with processing')
all_tweets_full = positive_tweets_full+negative_tweets_full #list of tweets
all_tweets_full, y_full = shuffle_tweets(all_tweets_full, y_full)


all loaded
finished with processing


In [ ]:
features = 200
epoch = 10
#print('let the word embedding commence')
#model_tot_sg1 = createWordEmbedding(all_tweets_full, features, epoch, sg=1) #word embedding
#model_tot_sg0 = createWordEmbedding(all_tweets_full, features, epoch) #word embedding
#print('word embedding created')
'''
features = 350 # we should find optimal value for this:
epoch = 10 # we should fin optimal value for this:
model_tot = createWordEmbedding(all_tweets, features, epoch) #word embedding
print('word embedding created')
'''

let the word embedding commence
1
word embedding created, start training:
0


In [ ]:
#model_tot_sg1.save('model_tot_sg1_200.word2vec')
#model_tot_sg0.save('model_tot_200.word2vec')

In [ ]:
model_tot = Word2Vec.load("model_tot_200.word2vec")
model_tot_sg1 = Word2Vec.load('model_tot_sg1_200.word2vec')

In [ ]:
X_old = create_X(all_tweets,model_tot, features)
X_old = preprocessing.scale(X_old)
print('processed feature matrix for tweets')

In [ ]:
#b = np.ones((X.shape[0], X.shape[1]+1)); b[:,:-1] = X
X = np.ones((X_old.shape[0],X_old.shape[1]+1))
X[:,1:] = X_old

In [ ]:
testd_tweets = open_file("test_data.txt")
testd = processTrainingData(testd_tweets)
print('loaded test file')
print(testd_tweets[:2])

In [ ]:
X_test_old = create_X(testd, model_tot, features)
X_test_old = preprocessing.scale(X_test_old)
X_test = np.ones((X_test_old.shape[0],X_test_old.shape[1]+1))
X_test[:,1:] = X_test_old
print('start training')

# Build logistic regression classifiers to identify the polarity of words
#test_y_lr = train(LogisticRegression(solver='lbfgs', max_iter = 1000), X, y, X_test)

# Build naive bayes classifiers to identify the polarity of words
#test_y_nb = train(nb.GaussianNB(), X, y, X_test) # this one isn't working

#print('created classification for submission')

#save_csv('test_resultLR.csv', test_y_lr)
#print('saved')

## Doing some cross validation:

In [ ]:
#clf = svm.SVC(gamma='scale')

#cv_results_clf = cross_validate(clf, X, y, return_train_score=False)
#print(cv_results_clf['test_score'])

#cv_results_rf = cross_validate(RandomForestClassifier(), X, y, return_train_score=False)
#print(cv_results_rf['test_score'])

cv_results_lr = cross_validate(LogisticRegression(solver = 'lbfgs'), X, y, return_train_score=False, cv = 5)
#print(cv_results_lr['test_score'])
print(np.mean(np.asarray(cv_results_lr['test_score'])))
#cv_results_nb = cross_validate(nb.GaussianNB(), X, y, return_train_score=False)
#print(cv_results_nb['test_score'])
print('done with cross validation')

In [ ]:
positive = open_file("train_pos.txt")
negative = open_file("train_neg.txt")

list_all_tweets = positive+negative
y_full = [1]*len(positive)+[0]*len(negative)
print(len(y))
list_all_tweets, y_full = shuffle_tweets(list_all_tweets, y_full)
list_all_tweets = easyProcess(list_all_tweets)
print('shuffled')

In [ ]:
SEED = 42
x_train, x_validation, y_train, y_validation = train_test_split(list_all_tweets, y_full, test_size=.1, random_state=SEED)

In [ ]:
print(len(x_train),len(y_train))

In [ ]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
from gensim.models import KeyedVectors
model_tot_k = KeyedVectors.load('model_tot.word2vec')
model_tot_k2 = KeyedVectors.load('model_tot_sg1.word2vec')

In [ ]:
len(model_tot_k.wv.vocab.keys())

In [ ]:
embeddings_index = {}
for w in model_tot_k.wv.vocab.keys():
    embeddings_index[w] = np.append(model_tot_k.wv[w],model_tot_k2.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
for x in x_train[:5]:
    print(x)

In [ ]:
print(len(x_train))

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [ ]:
print(len(x_train))

In [ ]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)
length = max(length)+2

In [ ]:
x_train_seq = pad_sequences(sequences, maxlen=length)
print('Shape of data tensor:', x_train_seq.shape)

In [ ]:

sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=length)

In [ ]:
num_words = 100000
embedding_matrix = np.zeros((num_words, features))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        

In [ ]:
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [ ]:
print(len(x_train), len(y_train), len(x_train_seq))

In [ ]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
hFeatures = features*0.5
structure_test = Sequential()
e = Embedding(100000, features, input_length=length)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

In [ ]:
model_cnn_03 = Sequential()
e = Embedding(100000, features, weights=[embedding_matrix], input_length=length, trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=features*0.5, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_03.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=3, batch_size=32, verbose=2)

In [ ]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

tweet_input = Input(shape=(length,), dtype='int32')

tweet_encoder = Embedding(100000, features, weights=[embedding_matrix], input_length=length, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=features*0.5, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=features*0.5, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=features*0.5, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=32, epochs=2,
                     validation_data=(x_val_seq, y_validation), callbacks = [checkpoint])

In [ ]:
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.02-0.8333.hdf5')
loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)

In [ ]:
testd_tweets = easyProcess(testd_tweets)

sequences_test = tokenizer.texts_to_sequences(testd_tweets)
x_test_seq = pad_sequences(sequences_test, maxlen=length)
ycnn01 = loaded_CNN_model.predict_classes(x=x_test_seq)
ycnn02 = loaded_CNN_model.predict(x=x_test_seq)

In [ ]:
positive = open_file("train_pos_full.txt")
negative = open_file("train_neg_full.txt")

list_all_tweets = positive+negative
y_full = [1]*len(positive)+[0]*len(negative)
print(len(y))
list_all_tweets, y_full = shuffle_tweets(list_all_tweets, y_full)
print('shuffled')

In [ ]:
tvec = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
print('start fitting')
tvec.fit(list_all_tweets)
print('done fitting')

In [ ]:
x_train_tfidf = tvec.transform(list_all_tweets)
x_test_tfidf = tvec.transform(testd_tweets)
print('start LR')
lr_with_tfidf = LogisticRegression(solver = 'lbfgs', max_iter = 1000)
lr_with_tfidf.fit(x_train_tfidf,y_full)
print('Predict LR')
testy = lr_with_tfidf.predict(x_test_tfidf)
save_csv('test_resulttfid.csv', testy)
print('saved')

In [ ]:
cv_results_lr = cross_validate(LogisticRegression(solver = 'lbfgs', max_iter = 500), x_train_tfidf, y_full, return_train_score=False, cv = 5)
#print(cv_results_lr['test_score'])
print(np.mean(np.asarray(cv_results_lr['test_score'])))